# **Redes Neurais**
## **Modelo 3 - Variational Autoencoder (VAE)**

- Aluno: Tales Miguel
- RA: 140247

- Professor Dr. Marcos G. Quiles

## **Instruções**

▸Selecionar 2 datasets (rotulados) -----> Se for um relatório detalhado, pode usar apenas 1 dataset

▸Treinar modelos VAEs:
  - Ajustar o melhor modelo (topologia) segundo a função de custo (conjunto validação)

▸Explorar o espaço latente:
  - Gerar gráficos com a projeção do espaço latente em 2D (PCA)
  - Usar os rótulos na projeção

▸Algumas questões:
  1. Há formação de clusters no espaço latente?
  2. Há separação dos rótulos no espaço latente?
  3. A projeção ilustra quanto da variância?

▸ Adicional (opcional): Enviesar a formação do espaço latente com os exemplos rotulados

## **1. Escolha do Dataset e Pré-processamento dos dados**

### **1.1. Dataset Escolhido - EMNIST Letters**

### **1.2. Imports e Pré-processamento**

### **1.3. Visualização Exploratória dos Dados**

## **2. Implementação do Variational Autoencoder (VAE)**

### **2.1. Configuração das Topologias para Experimentos**

### **2.2. Treinamento e Seleção da Melhor Topologia**

### **2.3. Análise Comparativa das Topologias**

## **3. Exploração do Espaço Latente**

### **3.1. Projeção do Espaço Latente em 2D (PCA)**

### **3.2. Questão 1: Há Formação de Clusters no Espaço Latente?**

### **3.3. Questão 2: Há Separação dos Rótulos no Espaço Latente?**

### **3.4. Questão 3: A Projeção Ilustra Quanto da Variância?**

### **3.5. Interpolação no Espaço Latente**

### **3.6. Geração de Novas Amostras**

## **4. Avaliação Final e Conclusões**

### **4.1. Avaliação no Conjunto de Teste**

### **4.2. Síntese dos Resultados e Conclusões**